# Load the fused fusion

In [1]:
# import pandas as pd

# 加载特征数据
# features_path = 'D:\\CAPSTONE5703_CNN\\final02.csv'
# features_df = pd.read_csv(features_path)

# 打印总行数和总列数
# print(f"总行数: {features_df.shape[0]}")
# print(f"总列数: {features_df.shape[1]}")

# 打印前五行数据
# print(features_df.head())

# Load the label 

In [2]:
# import pandas as pd
# import pickle

# def load_labels_from_pickle(pickle_file):
    # with open(pickle_file, 'rb') as f:
        # data = pickle.load(f)
        # labels = data['label']
    # return labels

# 初始化 PickleDataset
# pickle_path = 'D:\\CAPSTONE5703_CNN\\datasets_pickle\\train.pkl'
# labels = load_labels_from_pickle(pickle_path)

# 将标签转换为 DataFrame
# labels_df = pd.DataFrame({'label': labels})

# 打印标签总行数
# print(f"Total number of rows of labels: {len(labels_df)}")

# 打印前 5 行标签
# print("First 5 rows of labels:")
# print(labels_df.head())

# Data alignment - merge through 'image_id'

In [1]:
import pandas as pd
import pickle

def load_labels_from_pickle(pickle_file):
    with open(pickle_file, 'rb') as f:
        data = pickle.load(f)
        # 直接访问 data 字典中的 'image_id' 和 'label'
        image_ids = data['image_id']
        labels = data['label']
        labels_dict = dict(zip(image_ids, labels))
    return labels_dict

# 加载 pickle 文件
pickle_path = 'train.pkl'
labels_dict = load_labels_from_pickle(pickle_path)

# 加载 CSV 文件
features_path = 'fused.csv'
features_df = pd.read_csv(features_path)

# 确保 CSV 文件中最后一列为 image_id
if features_df.columns[-1] != 'image_id':
    print("Error: 'image_id' must be the last column in the CSV file.")
else:
    # 创建 DataFrame 用于包含从字典中提取的 image_id 和 label
    labels_df = pd.DataFrame(list(labels_dict.items()), columns=['image_id', 'label'])

    # 将 features_df 和 labels_df 中的 'image_id' 设置为索引
    features_df.set_index('image_id', inplace=True)
    labels_df.set_index('image_id', inplace=True)

    # 根据 image_id 合并 features_df 和 labels_df
    merged_df = pd.merge(features_df, labels_df, left_index=True, right_index=True, how='inner')

    # 重置索引以便导出或其他处理
    merged_df.reset_index(inplace=True)

    # 检查合并后的数据
    print(merged_df.head())

    # 可以选择保存合并后的 DataFrame
    # merged_df.to_csv('D:\\CAPSTONE5703_CNN\\merged_fused features.csv', index=False)


                           image_id         0         1         2         3  \
0  62b31d36gw1expsi2gfrdj20hm0loq8o -0.521330  0.065525 -0.028026  0.187444   
1  563a2b53jw1exl77nkup7j20c30f3q4j -0.482425  0.079567 -0.036039  0.194375   
2  005ldo0ygw1ex23rdfuqcj30xo0k6di0 -0.468386  0.084869 -0.049615  0.144983   
3  62b31d36gw1exfcmyz8agj20qq0hu77k -0.468805  0.067019 -0.031303  0.199792   
4  0060kjm0jw1exdjaeiqadj30xc0m8tdw -0.473862  0.089600 -0.023923  0.144352   

          4         5         6         7         8  ...      1527      1528  \
0  0.023185  0.102659 -0.126628 -0.045669  0.036374  ... -0.239570 -0.247390   
1  0.036827  0.083796 -0.112961  0.000671 -0.050403  ... -0.178980 -0.252651   
2  0.038398  0.096167 -0.096763 -0.018022  0.016321  ... -0.167772 -0.270531   
3  0.033397  0.084818 -0.119774 -0.028753 -0.030460  ... -0.165330 -0.279122   
4  0.033620  0.057612 -0.104852 -0.013622 -0.017427  ... -0.209840 -0.229515   

       1529      1530      1531      1532   

# Split training & test set

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 从DataFrame中分离出特征数据和标签
X = merged_df.drop(['label', 'image_id'], axis=1)  # 移除 'label' 和 'image_id' 列
y = merged_df['label']  # 标签

# 打印出每列的数据类型
# print(X.dtypes)

# 将数据集划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train and X_test represent the training set and test set of feature data respectively.
# y_train and y_test represent the training set and test set of labeled data respectively.





# PCA 

In [3]:
import numpy as np
from sklearn.decomposition import PCA

# 应用 PCA 降维，只在训练数据上fit，然后transform训练数据和测试数据
pca = PCA(n_components=0.95)  # 保留95%的方差
X_train_pca = pca.fit_transform(X_train) # pca后的训练集
X_test_pca = pca.transform(X_test) # pca后的测试集

# 检查新的维数和解释的方差比
print("New training dimensions:", X_train_pca.shape[1])
print("New testing dimensions:", X_test_pca.shape[1])
# print("Explained variance ratio:", pca.explained_variance_ratio_)


New training dimensions: 193
New testing dimensions: 193


# MLP - scikit-learn

## 2-hidden

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import RandomizedSearchCV


# 创建并配置MLP分类器
mlp = MLPClassifier(random_state=42,shuffle=True,tol=1e-4,early_stopping=True, n_iter_no_change=10)

# 定义需要优化的超参数和对应的值
param_distributions = {
    'hidden_layer_sizes': [(200, 100), (300, 150), (100, 50)],  
    'activation': ['logistic','relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001,0.001, 0.01],
    'learning_rate_init': [0.0001, 0.001, 0.01, 0.1, 1],
    'max_iter': [500,800,1100],
    'batch_size': [64]
}

# 创建RandomizedSearchCV对象
random_search = RandomizedSearchCV(mlp, param_distributions=param_distributions, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, n_iter=40)
random_search.fit(X_train_pca, y_train)  # 使用降维后的训练数据

# 打印最佳超参数和对应的分数
print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

# 使用最佳超参数创建新的MLP模型
best_mlp = random_search.best_estimator_

# 在完整的训练集上训练最佳模型
best_mlp.fit(X_train_pca, y_train)

# 在测试集上评估最佳模型
predictions = best_mlp.predict(X_test_pca)
print("Accuracy on test set: ", accuracy_score(y_test, predictions))
print("\nClassification Report:\n", classification_report(y_test, predictions))


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best parameters:  {'solver': 'adam', 'max_iter': 800, 'learning_rate_init': 0.1, 'hidden_layer_sizes': (200, 100), 'batch_size': 64, 'alpha': 0.001, 'activation': 'relu'}
Best score:  0.5863267192490365
Accuracy on test set:  0.6131117266851339

Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.59      0.59       517
           1       0.63      0.63      0.63       566

    accuracy                           0.61      1083
   macro avg       0.61      0.61      0.61      1083
weighted avg       0.61      0.61      0.61      1083



## 3 hidden

In [5]:
mlp = MLPClassifier(random_state=42,shuffle=True,tol=1e-4,early_stopping=True, n_iter_no_change=10)

# 定义需要优化的超参数和对应的值
param_distributions = {
    'hidden_layer_sizes': [(200,100,50), (300,150,75)],  
    'activation': ['logistic','relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001,0.001, 0.01],
    'learning_rate_init': [0.0001, 0.001, 0.01, 0.1, 1],
    'max_iter': [500,800,1100],
    'batch_size': [64]
}

# 创建RandomizedSearchCV对象
random_search = RandomizedSearchCV(mlp, param_distributions=param_distributions, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, n_iter=40)
random_search.fit(X_train_pca, y_train)  # 使用降维后的训练数据

# 打印最佳超参数和对应的分数
print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

# 使用最佳超参数创建新的MLP模型
best_mlp = random_search.best_estimator_

# 在完整的训练集上训练最佳模型
best_mlp.fit(X_train_pca, y_train)

# 在测试集上评估最佳模型
predictions = best_mlp.predict(X_test_pca)
print("Accuracy on test set: ", accuracy_score(y_test, predictions))
print("\nClassification Report:\n", classification_report(y_test, predictions))



Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best parameters:  {'solver': 'sgd', 'max_iter': 1100, 'learning_rate_init': 0.1, 'hidden_layer_sizes': (300, 150, 75), 'batch_size': 64, 'alpha': 0.001, 'activation': 'relu'}
Best score:  0.573407278955598
Accuracy on test set:  0.5872576177285319

Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.71      0.62       517
           1       0.64      0.47      0.55       566

    accuracy                           0.59      1083
   macro avg       0.60      0.59      0.58      1083
weighted avg       0.60      0.59      0.58      1083



## 4 hidden

In [6]:

mlp = MLPClassifier(random_state=42,shuffle=True,tol=1e-4,early_stopping=True, n_iter_no_change=10)

# 定义需要优化的超参数和对应的值
param_distributions = {
    'hidden_layer_sizes': [(200,100,50,25), (300,150,75,30)],  
    'activation': ['logistic','relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001,0.001, 0.01],
    'learning_rate_init': [0.0001, 0.001, 0.01, 0.1, 1],
    'max_iter': [500,800,1100],
    'batch_size': [64]
}


# 创建RandomizedSearchCV对象
random_search = RandomizedSearchCV(mlp, param_distributions=param_distributions, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, n_iter=40)
random_search.fit(X_train_pca, y_train)  # 使用降维后的训练数据

# 打印最佳超参数和对应的分数
print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

# 使用最佳超参数创建新的MLP模型
best_mlp = random_search.best_estimator_

# 在完整的训练集上训练最佳模型
best_mlp.fit(X_train_pca, y_train)

# 在测试集上评估最佳模型
predictions = best_mlp.predict(X_test_pca)
print("Accuracy on test set: ", accuracy_score(y_test, predictions))
print("\nClassification Report:\n", classification_report(y_test, predictions))


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best parameters:  {'solver': 'adam', 'max_iter': 800, 'learning_rate_init': 0.001, 'hidden_layer_sizes': (200, 100, 50, 25), 'batch_size': 64, 'alpha': 0.001, 'activation': 'relu'}
Best score:  0.5888753925697436
Accuracy on test set:  0.6214219759926131

Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.59      0.60       517
           1       0.63      0.65      0.64       566

    accuracy                           0.62      1083
   macro avg       0.62      0.62      0.62      1083
weighted avg       0.62      0.62      0.62      1083



# SVM

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score

# 将数据重塑为二维
X_train_pca_reshaped = X_train_pca.reshape(X_train_pca.shape[0], X_train_pca.shape[2])
X_test_pca_reshaped = X_test_pca.reshape(X_test_pca.shape[0], X_test_pca.shape[2])

# 创建SVM模型，尝试使用RBF核
svc = SVC(kernel='rbf')

# 设置更广泛的参数网格
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.01, 0.1, 1, 'scale']
}

# 使用网格搜索进行调参，应用StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(svc, param_grid, refit=True, verbose=2, cv=cv)
random_search.fit(X_train_pca_reshaped, y_train)

# 预测
predictions = random_search.predict(X_test_pca_reshaped)

# 评估模型
print("Best parameters found:", random_search.best_params_)
print("Accuracy:", accuracy_score(y_test, predictions))
print("\nClassification Report:\n", classification_report(y_test, predictions))


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ...................................C=10, gamma=0.01; total time=   2.3s
[CV] END ...................................C=10, gamma=0.01; total time=   2.0s
[CV] END ...................................C=10, gamma=0.01; total time=   2.0s
[CV] END ...................................C=10, gamma=0.01; total time=   2.0s
[CV] END ...................................C=10, gamma=0.01; total time=   2.1s
[CV] END .....................................C=100, gamma=1; total time=   2.5s
[CV] END .....................................C=100, gamma=1; total time=   2.6s
[CV] END .....................................C=100, gamma=1; total time=   2.5s
[CV] END .....................................C=100, gamma=1; total time=   2.5s
[CV] END .....................................C=100, gamma=1; total time=   2.5s
[CV] END ....................................C=1, gamma=0.01; total time=   2.1s
[CV] END ....................................C=1

# Random Forest 

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 将数据重塑为二维
X_train_pca_reshaped = X_train_pca.reshape(X_train_pca.shape[0], X_train_pca.shape[2])
X_test_pca_reshaped = X_test_pca.reshape(X_test_pca.shape[0], X_test_pca.shape[2])

# 设置参数分布
param_dist = {
    'n_estimators': [100, 200, 300],  # 树的数量
    'max_depth': [None, 10, 20, 30],  # 树的最大深度
    'min_samples_split': [2, 5, 10],  # 内部节点再划分所需最小样本数
    'min_samples_leaf': [1, 2, 4],    # 叶子节点最小样本数
    'bootstrap': [True, False]        # 是否有放回地抽样
}

# 初始化随机森林分类器
rf = RandomForestClassifier(random_state=42)

# 设置K折交叉验证
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 运行随机搜索
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=40, cv=cv, scoring='accuracy', n_jobs=-1, random_state=42, verbose=1)
random_result = random_search.fit(X_train_pca_reshaped, y_train)

# 输出结果
print("Best parameters found:", random_result.best_params_)
print("Best score:", random_result.best_score_)

# 使用最佳参数训练模型并评估在测试集上的性能
best_rf = random_result.best_estimator_
best_rf.fit(X_train_pca_reshaped, y_train)
y_pred = best_rf.predict(X_test_pca_reshaped)
test_accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy on test set: {test_accuracy}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best parameters found: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': None, 'bootstrap': False}
Best score: 0.6075775616590884
Accuracy on test set: 0.6112650046168052

Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.63      0.61       517
           1       0.64      0.59      0.61       566

    accuracy                           0.61      1083
   macro avg       0.61      0.61      0.61      1083
weighted avg       0.61      0.61      0.61      1083



# MLP - design baseline 

## 4 hidden -BCEWithLogitsLoss - Xavier


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from torch.utils.data import Dataset, DataLoader, TensorDataset

# 重塑数据为二维
X_train_pca_reshaped = X_train_pca.reshape(X_train_pca.shape[0], X_train_pca.shape[2])
X_test_pca_reshaped = X_test_pca.reshape(X_test_pca.shape[0], X_test_pca.shape[2])

# 准备数据
X_train_tensor = torch.from_numpy(X_train_pca_reshaped).float()
y_train_tensor = torch.from_numpy(y_train).float()
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 初始化模型
class MLP(nn.Module):
    def __init__(self, input_size=193):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(input_size, 200)
        self.prelu1 = nn.ReLU()
        self.layer2 = nn.Linear(200, 100)
        self.prelu2 = nn.ReLU()
        self.layer3 = nn.Linear(100, 50)
        self.prelu3 = nn.ReLU()
        self.layer4 = nn.Linear(50, 25)
        self.prelu4 = nn.ReLU()
        self.output_layer = nn.Linear(25, 1)
        self.init_weights()

    def init_weights(self):
        for layer in self.children():
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)  # Xavier 初始化
                if layer.bias is not None:
                    nn.init.constant_(layer.bias, 0)

    def forward(self, x):
        x = self.prelu1(self.layer1(x))
        x = self.prelu2(self.layer2(x))
        x = self.prelu3(self.layer3(x))
        x = self.prelu4(self.layer4(x))
        x = self.output_layer(x)
        return x

model = MLP(input_size=193)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)  # 使用Adam优化器

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum / y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc.item()

# 开始训练
start_time = time.time()
best_acc = 0

for epoch in range(200):  # 训练200个epoch
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):  
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()
    
    # 计算训练集上的准确率
    model.eval()
    with torch.no_grad():
        train_outputs = model(X_train_tensor)
        train_acc = binary_acc(train_outputs, y_train_tensor.view(-1, 1))
        if train_acc > best_acc:
            best_acc = train_acc

end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds")
print(f"Best Training Accuracy = {best_acc}")

# 模型评估
model.eval()  # 将模型设置为评估模式
with torch.no_grad():  # 禁止跟踪计算图
    X_test_tensor = torch.from_numpy(X_test_pca_reshaped).float() 
    y_test_tensor = torch.from_numpy(y_test).float()  # 将 NumPy 数组转换为 PyTorch 张量，并转换数据类型为浮点型
    test_outputs = model(X_test_tensor)
    test_acc = binary_acc(test_outputs, y_test_tensor.view(-1, 1))
    print(f"Accuracy on test set: {test_accuracy}")
    print("\nClassification Report:\n", classification_report(y_test_tensor, y_pred))


Training completed in 246.93 seconds
Best Training Accuracy = 99.0
Accuracy on test set: 0.6112650046168052

Classification Report:
               precision    recall  f1-score   support

         0.0       0.59      0.63      0.61       517
         1.0       0.64      0.59      0.61       566

    accuracy                           0.61      1083
   macro avg       0.61      0.61      0.61      1083
weighted avg       0.61      0.61      0.61      1083



## 5 hidden - without PCA 

In [ ]:

mlp = MLPClassifier(random_state=42,shuffle=True,tol=1e-4,early_stopping=True, n_iter_no_change=10)

# 定义需要优化的超参数和对应的值
param_distributions = {
    'hidden_layer_sizes': [(1536,768,384,192,96)],  
    'activation': ['logistic','relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001,0.001, 0.01],
    'learning_rate_init': [0.0001, 0.001, 0.01, 0.1, 1],
    'max_iter': [800,1100,1400],
    'batch_size': [64]
}

# 创建RandomizedSearchCV对象
random_search = RandomizedSearchCV(mlp, param_distributions=param_distributions, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, n_iter=10)
random_search.fit(X_train, y_train)  # 使用降维后的训练数据

# 打印最佳超参数和对应的分数
print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

# 使用最佳超参数创建新的MLP模型
best_mlp = random_search.best_estimator_

# 在完整的训练集上训练最佳模型
best_mlp.fit(X_train, y_train)

# 在测试集上评估最佳模型
predictions = best_mlp.predict(X_test)
print("Accuracy on test set: ", accuracy_score(y_test, predictions))
print("\nClassification Report:\n", classification_report(y_test, predictions))
